# Scraping Anime from Zoro.to

In [ ]:

# Importing libraries

import requests
from bs4 import BeautifulSoup
import pandas as pd

page_urls = []

for i in range(1,170):
    if i == 1:
        page_urls.append('https://aniwatch.to/az-list/')
    else:
        page_urls.append('https://aniwatch.to/az-list/?page=' + str(i) + '/')
        pass
    pass
page_urls
# Scraping the data from all the pages

anime_urls = []

for url in page_urls:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Getting the url for the anime page
    
    for anime in soup.find_all('div',class_='film-poster'):
        anime = anime.find('a')['href']
        page = 'https://aniwatch.to'+ anime
        anime_urls.append(page)
        pass
    

    pass
anime_dict = {'anime_urls':anime_urls}
anime_df = pd.DataFrame(anime_dict)

anime_df.to_csv('anime_urls.csv',index=False)


# Scraping the data from all the anime pages

In [ ]:
import warnings as w
w.filterwarnings('ignore')

df = pd.read_csv('anime_urls.csv')
df.head()

In [ ]:
def create_df(df):

    anime_poster = []
    anime_title = []
    anime_overview = []
    anime_mal_score = []
    anime_views = []
    anime_genres = []
    anime_studio = []
    anime_producer = []

    for url in df['anime_urls']:
        
        soup = BeautifulSoup(requests.get(url).content, 'html.parser')
        
        # Getting the poster of the anime (url)
        
        anime_poster.append(soup.find('div',class_='film-poster').find('img')['src'])

        # Getting the name of the anime
        
        anime_title.append(soup.find('h2',class_='film-name dynamic-name').text)
        
        # Getting the overview of the anime
        
        anime_overview.append(soup.find('div',class_='item item-title w-hide').find('div',class_='text').text)
        
        # Creating an object of the div containing all the details of the anime
        
        soup = soup.find('div',class_='anisc-info')

        # Extract MAL Score
        mal_score_element = soup.find('span', {'class': 'item-head'}, text='MAL Score:')
        mal_score = mal_score_element.find_next_sibling('span', {'class': 'name'}).text.strip() if mal_score_element else 'NA'

        # Extract Views
        views_element = soup.find('span', {'class': 'item-head'}, text='Views:')
        views = views_element.find_next_sibling('span', {'class': 'name'}).text.strip() if views_element else 'NA'

        # Extract Studios
        studios_element = soup.find('span', {'class': 'item-head'}, text='Studios:')
        studios = studios_element.find_next('a', {'class': 'name'}).text.strip() if studios_element else 'NA'

        # Extract Producers
        producers_element = soup.find('span', {'class': 'item-head'}, text='Producers:')
        producers = [producer.text.strip() for producer in producers_element.find_next_siblings('a')] if producers_element else ['NA']

        # Extract Genres
        genres_element = soup.find('span', {'class': 'item-head'}, text='Genres:')
        genres = [genre.text.strip() for genre in genres_element.find_next_siblings('a')] if genres_element else ['NA']

        anime_mal_score.append(mal_score)
        anime_views.append(views)
        anime_studio.append(studios)
        anime_producer.append(producers)
        anime_genres.append(genres)
        pass
    
    anime_dict = {'anime_poster':anime_poster,'anime_title':anime_title,'anime_overview':anime_overview,'anime_mal_score':anime_mal_score,'anime_views':anime_views,'anime_studio':anime_studio,'anime_producer':anime_producer,'anime_genres':anime_genres}
    anime_df = pd.DataFrame(anime_dict)
    return anime_df


In [ ]:
anime_df = create_df(df)
anime_df.head()

In [ ]:
anime_df.to_csv('anime_data.csv',index=False)